In [1]:
import requests#Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy#authentication
import spotipy.util as util#authentication
from spotipy.oauth2 import SpotifyClientCredentials#authentication

In [2]:
#Credentials
cid = '049ade7215e54c63a2b628f3784dc407'
secret = 'b86bedbebf53486b970e92b2cb7b612c'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
def get_album_track(url,artist_uri):
    all_albums = []
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
        
    response = resp.json()
    
    album_limit = (response['limit'])-1
    
    for x in range(album_limit):
        try:
            track_uri = response['items'][x]['uri'].split(':')
            track_name = response['items'][x]['name'].replace('[','').replace(']','')
            print(track_uri,track_name)
            all_albums.append({'track_uri':track_uri[2],'track_name':track_name})
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
    try:
        if (url is not None):
            get_albums(response['next'])
        else:
            return
    except:
        return all_albums

In [5]:
i = 0
track_list=[]

start = time.time()
with open('Data/artist_album_uri.csv') as csvfile:
    file = csv.reader(csvfile, delimiter='|')
    next(file)
    head = [next(file) for x in range(1)] #for limiting amount of rows read
    
    for row in head:
        artist_uri = row[0]
        artist_name = row[1]
        album_uri = row[2]
        print(i,artist_uri,artist_name,album_uri)
        albumtracks = get_album_track('https://api.spotify.com/v1/albums/{id}/tracks?limit=14'.format(id=album_uri),artist_uri)#list of album uris
        for track in albumtracks:
            track_list.append({'Artist URI':artist_uri,'Artist Name':artist_name,'Album URI':album_uri
                               ,'track_uri':track['track_uri'],'track_name':track['track_name']})#output album_uri
        
        i+=1
        
stop = time.time()
duration = (stop - start)/60

0 3bBWKHfpepPOychRNFzg4q al di meola 7xbyqbmIudUFYHsgHEvcwp
['spotify', 'track', '7cVA2h2HRlE2mOxhXEmtro'] Here Comes the Sun
['spotify', 'track', '5N6IdYnIYaATgQKHg9CZJP'] Golden Slumbers
['spotify', 'track', '2Fz3MpPkiUPEH6d4Heuipp'] Dear Prudence
['spotify', 'track', '4yDo6bylEHQMbAaXLKSvwl'] Norwegian Wood
['spotify', 'track', '1kgKtxtHLvRPrmvX8ns79t'] Mother Nature's Son
['spotify', 'track', '2f9m1gEeEveK3E4nkXOvD3'] Strawberry Fields Forever
['spotify', 'track', '1yaj9IUWz7QaBpuwYxRsp1'] Yesterday
['spotify', 'track', '1M9aXWCWRtmiuUJuDNMUyg'] Your Mother Should Know
['spotify', 'track', '2V3KpCCtGjjCJ1MUnKid89'] Hey Jude
['spotify', 'track', '3tYOJ81C6qHIIkW8FAU69e'] I'll Follow the Sun
['spotify', 'track', '2wBMA1jDkLAwcotlwgRATP'] Julia
['spotify', 'track', '3hmkXA8pT2eT08wVM8md28'] Till There Was You
['spotify', 'track', '2WWXgrorazoGDKb4NJ0fMJ'] Here, There and Everywhere


In [6]:
track_list

[{'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'al di meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '7cVA2h2HRlE2mOxhXEmtro',
  'track_name': 'Here Comes the Sun'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'al di meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '5N6IdYnIYaATgQKHg9CZJP',
  'track_name': 'Golden Slumbers'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'al di meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '2Fz3MpPkiUPEH6d4Heuipp',
  'track_name': 'Dear Prudence'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'al di meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '4yDo6bylEHQMbAaXLKSvwl',
  'track_name': 'Norwegian Wood'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'al di meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '1kgKtxtHLvRPrmvX8ns79t',
  'track_name': "Mother Nature's Son"},
 {'Artist URI': '3bBWKHfpepPOychRNFz

In [7]:
#Duration of script
str(duration)+' minutes'

'0.0022235711415608725 minutes'

In [8]:
df = pd.DataFrame(track_list)
df
#Test whether there are duplicates for Album URI
#df_dropduplicates = df.drop_duplicates([''])
#df_dropduplicates
#df_dropduplicates.to_csv('Data/artist_album_track_uri.csv',sep='|',index=False)

,Artist URI,Artist Name,Album URI,track_uri,track_name
0,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,7cVA2h2HRlE2mOxhXEmtro,Here Comes the Sun
1,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,5N6IdYnIYaATgQKHg9CZJP,Golden Slumbers
2,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,2Fz3MpPkiUPEH6d4Heuipp,Dear Prudence
3,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,4yDo6bylEHQMbAaXLKSvwl,Norwegian Wood
4,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,1kgKtxtHLvRPrmvX8ns79t,Mother Nature's Son
5,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,2f9m1gEeEveK3E4nkXOvD3,Strawberry Fields Forever
6,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,1yaj9IUWz7QaBpuwYxRsp1,Yesterday
7,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,1M9aXWCWRtmiuUJuDNMUyg,Your Mother Should Know
8,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,2V3KpCCtGjjCJ1MUnKid89,Hey Jude
9,3bBWKHfpepPOychRNFzg4q,al di meola,7xbyqbmIudUFYHsgHEvcwp,3tYOJ81C6qHIIkW8FAU69e,I'll Follow the Sun
